### Example 0 Vecadd

In [13]:
import taichi as ti
ti.init(arch=ti.cpu)

input_a = ti.ndarray(dtype=ti.f32, shape=(10000))
input_b = ti.ndarray(dtype=ti.f32, shape=(10000))
output_c = ti.ndarray(dtype=ti.f32, shape=(10000))

@ti.kernel
def vecadd(a:ti.types.ndarray(dtype=ti.f32, ndim=1), \
           b:ti.types.ndarray(dtype=ti.f32, ndim=1), \
           c:ti.types.ndarray(dtype=ti.f32, ndim=1)):
    for ii in range(10000):
        a[ii] = ii
        b[ii] = ii
        c[ii] = a[ii] + b[ii] + c[ii]


vecadd(input_a, input_b, output_c)

[Taichi] Starting on arch=x64


### Example 1
初识Taichi，使用Taichi筛素数

In [15]:
import taichi as ti
import time
ti.init(arch=ti.cpu, kernel_profiler=True)
@ti.func
def is_prime(n: int):
   result = True
   for k in range(2, int(n ** 0.5) + 1):
       if n % k == 0:
           result = False
           break
   return result
@ti.kernel
def count_primes(n: int) -> int:
   count = 0
   for k in range(2, n):
       if is_prime(k):
           count += 1
   return count
#start_time = time.time()
print(count_primes(1000000))
# Taichi 1000000
#    CPU 0.015s
#    GPU 0.025s
# Python 1000000
#    CPU 3.48s
#print(time.time()-start_time)
ti.profiler.print_kernel_profiler_info()

[Taichi] Starting on arch=x64
78498
Kernel Profiler(count, default) @ X64 
[      %     total   count |      min       avg       max   ] Kernel name
-------------------------------------------------------------------------
[ 99.96%   0.015 s      1x |   14.851    14.851    14.851 ms] count_primes_c92_0_kernel_2_range_for
[  0.04%   0.000 s      1x |    0.006     0.006     0.006 ms] count_primes_c92_0_kernel_0_serial
[  0.00%   0.000 s      1x |    0.000     0.000     0.000 ms] count_primes_c92_0_kernel_1_serial
-------------------------------------------------------------------------
[100.00%] Total execution time:   0.015 s   number of results: 3


### Example2 AOS vs SOA
1. 如果有一些操作只用到属性的子集（比如说x, y, z 而没有 w），那么 SOA 的内存布局可以完全不浪费 w 造成的内存带宽。

    注：这里要注意的是 CPU 上的 cacheline 大小是 64B、(NVIDIA) GPU 上是128 B，内存带宽的瓶颈往往是 last-level cache (LLC，比如 CPU 上的 L3 cache、GPU 上的 L2 cache) 到 main memory（也就是内存条）的 bandwidth。LLC 到 main memory 的单位是 cacheline (而不是 byte)。如果一个 cacheline 只用一部分，就会导致 main memory bandwidth 的浪费。在 AOS 的 data layout 下，由于一个粒子的 x, y, z, w 总是在同一个 cacheline 内，没有办法只向 main memory fetch 其中的一部分。后续的会有图片来说明这一点。

2. 在 GPU 上，SOA 能够发挥 GPU 硬件的 Coalescing 机制，硬件会把一个 warp （32 个 thread）的数据访问打包成一个 memory transaction，提高效率。类似地，在 CPU 上，SOA 对于 SIMD 模式的编程也更加友好（如 x86 架构下可以使用 _mm_load_ps 等 SIMD intrinsics 对数据进行向量化加载）。

3. SOA 的另一个好处是比较容易在运行时给一组数据添加一个 attribute，并且保持访问的高效。本文就不针对这一点展开了。
抄自：https://zhuanlan.zhihu.com/p/552884861（强烈推荐大家学习原文章）

In [10]:
import taichi as ti

ti.init(arch=ti.cpu, kernel_profiler=True)

n = 64 * 1024 * 1024
dim = 4
unroll = 8

a = ti.Vector.field(dim, dtype=ti.i32, shape=n, 
                    layout=ti.Layout.SOA)

@ti.kernel
def assign_all():
    for i in range(n):
        for k in ti.static(range(dim)):
            a[i][k] = i + k
            
@ti.kernel
def assign_single():
    for i in range(n):
        a[i][0] = i
            
@ti.kernel
def assign_all_random():
    for i_ in range(n):
        for k in ti.static(range(dim)):
            i = (i_ * 10007) & (n - 1) # Random index
            a[i][k] = i + k
    
for repeat in range(30):
    assign_all()


for repeat in range(30):
    assign_single()

    
for repeat in range(30):
    assign_all_random()

    
ti.profiler.print_kernel_profiler_info()

[Taichi] Starting on arch=x64
Kernel Profiler(count, default) @ X64 
[      %     total   count |      min       avg       max   ] Kernel name
-------------------------------------------------------------------------
[ 92.28%  14.119 s     30x |  466.911   470.618   496.956 ms] assign_all_random_c110_0_kernel_0_range_for
[  4.54%   0.695 s     30x |   22.311    23.173    34.396 ms] assign_all_c106_0_kernel_0_range_for
[  3.18%   0.486 s     30x |   15.961    16.193    17.635 ms] assign_single_c108_0_kernel_0_range_for
-------------------------------------------------------------------------
[100.00%] Total execution time:  15.300 s   number of results: 3


|  Times(s)   | assign_all  | assign_single | assigen_all_random |
|  ----  | ---- | ---- | ---- | 
| AOS  | 0.68 | 0.61 | 8.37 |
| SOA  | 0.70 | 0.48 | 14.37 |

1. 不论是何种布局，顺序访问都比随机访问快。
2. assign_single任务下，SOA的性能要优于AOS
3. 如果你的访问是顺序访问，那么 SOA 通常比 AOS 效率高一些。特别是当你的程序在 GPU 上运行的时候，因为 Coalescing。
4. 如果你的访问是随机访问，那么 AOS 常常比 SOA 更好。因为随机访问的一个粒子的数据在 AOS 下是在同一个 cacheline 中，而在 SOA 布局下会分散在 4 个 cachelines 里面，对于 AOS 的 cacheline utilization（缓存行利用率）会高很多。比如在上面的例子中，assign_all_random 在 AOS 布局下就比 SOA 快了近 2 倍。

### Example3 Taichi-Sparse

In [12]:
from taichi.examples.patterns import taichi_logo

import taichi as ti

ti.init(arch=ti.cpu, offline_cache=False)

n = 512
x = ti.field(dtype=ti.i32)
res = n + n // 4 + n // 16 + n // 64
img = ti.field(dtype=ti.f32, shape=(res, res))

block1 = ti.root.pointer(ti.ij, n // 64) # 8 times 8 pointers
block2 = block1.pointer(ti.ij, 4)
block3 = block2.pointer(ti.ij, 4)
block3.dense(ti.ij, 4).place(x)


@ti.kernel
def activate(t: ti.f32):
    for i, j in ti.ndrange(n, n):
        p = ti.Vector([i, j]) / n
        p = ti.math.rotation2d(ti.sin(t)) @ (p - 0.5) + 0.5

        if taichi_logo(p) == 0:
            x[i, j] = 1


@ti.func
def scatter(i):
    return i + i // 4 + i // 16 + i // 64 + 2


@ti.kernel
def paint():
    for i, j in ti.ndrange(n, n):
        t = x[i, j]
        block1_index = ti.rescale_index(x, block1, [i, j])
        block2_index = ti.rescale_index(x, block2, [i, j])
        block3_index = ti.rescale_index(x, block3, [i, j])
        t += ti.is_active(block1, block1_index)
        t += ti.is_active(block2, block2_index)
        t += ti.is_active(block3, block3_index)
        img[scatter(i), scatter(j)] = 1 - t / 4


def main():
    img.fill(0)

    gui = ti.GUI('Sparse Grids', (res, res))

    for i in range(100000):
        block1.deactivate_all()
        activate(i * 0.05)
        paint()
        gui.set_image(img)
        gui.show()


if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


RuntimeError: Window close button clicked, exiting... (use `while gui.running` to exit gracefully)